In [1]:
"""Week 1: Core Tools and Data Access functions for geospatial AI."""

import sys
import importlib.metadata
import warnings
import os
from typing import Dict, List, Tuple, Optional, Union
from pathlib import Path
import time
import logging

# Core geospatial libraries
import rasterio
from rasterio.windows import from_bounds
from rasterio.warp import transform_bounds
import numpy as np
import pandas as pd
from pystac_client import Client
import planetary_computer as pc

warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def verify_environment(required_packages: list) -> dict:
    """
    Verify that all required packages are installed.

    Parameters
    ----------
    required_packages : list
        List of package names to verify

    Returns
    -------
    dict
        Dictionary with package names as keys and versions as values
    """
    results = {}
    missing_packages = []

    for package in required_packages:
        try:
            version = importlib.metadata.version(package)
            results[package] = version
        except importlib.metadata.PackageNotFoundError:
            missing_packages.append(package)
            results[package] = None

    # Report results
    if missing_packages:
        logger.error(f"❌ Missing packages: {', '.join(missing_packages)}")
        return results

    logger.info(f"✅ All {len(required_packages)} packages verified")
    return results

In [2]:
# Verify core geospatial AI environment
required_packages = [
    'rasterio', 'xarray', 'torch', 'transformers',
    'folium', 'matplotlib', 'numpy', 'pandas',
    'pystac-client', 'geopandas', 'rioxarray', 'planetary-computer'
]

package_status = verify_environment(required_packages)

2025-10-03 09:20:02,215 - INFO - ✅ All 12 packages verified


In [3]:
## Import Necessary Libraries

# Core geospatial libraries
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import xarray as xr
import rioxarray  # Extends xarray with rasterio functionality

# Data access and processing
import numpy as np
import pandas as pd
import geopandas as gpd
from pystac_client import Client
import planetary_computer as pc  # For signing asset URLs

# Visualization
import matplotlib.pyplot as plt
import folium
from folium import plugins

# Utilities
from typing import Dict, List, Tuple, Optional, Union
from pathlib import Path
import json
import time
from datetime import datetime, timedelta
import logging

# Deep learning libraries
import torch

In [4]:
# Configure matplotlib for publication-quality plots
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 100,
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9
})

In [5]:
# Send print statements away if dont want them for debugging
# Seems pretty helpful! 
# Info is a level - debug is everything, different levels here
# Check out this document 

# Configure logging for production-ready code
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

Create a function to set up the MPC

In [7]:
# telling it should return true or false, make sure to enforce this
# type checker (python not typed language), javascript mostly types, python getting with the times

def setup_planetary_computer_auth() -> bool:
    """
    Configure authentication for Microsoft Planetary Computer.

    Uses environment variables and .env files for credential discovery,
    with graceful degradation to anonymous access.

    Returns
    -------
    bool
        True if authenticated, False for anonymous access
    """
    # Try environment variables first (production)
    auth_key = os.getenv('PC_SDK_SUBSCRIPTION_KEY') or os.getenv('PLANETARY_COMPUTER_API_KEY')

    # Fallback to .env file (development)
    if not auth_key:
        env_file = Path('.env')
        if env_file.exists():
            try:
                with open(env_file) as f:
                    for line in f:
                        line = line.strip()
                        if line.startswith(('PC_SDK_SUBSCRIPTION_KEY=', 'PLANETARY_COMPUTER_API_KEY=')):
                            auth_key = line.split('=', 1)[1].strip().strip('"\'')
                            break
            except Exception:
                pass  # Continue with anonymous access

    # Configure authentication
    if auth_key and len(auth_key) > 10:
        try:
            pc.set_subscription_key(auth_key)
            logger.info("Planetary Computer authentication successful")
            return True
        except Exception as e:
            logger.warning(f"Authentication failed: {e}")

    logger.info("Using anonymous access (basic rate limits)")
    return False

In [8]:
# Initialize authentication
auth_status = setup_planetary_computer_auth()
# logger object exsists in python interpretor - running in background, might need to like destroy it?
# would require kernal restart for logger
logger.info(f"Planetary Computer authentication status: {'Authenticated' if auth_status else 'Anonymous'}")

2025-10-03 09:30:04,417 - INFO - Using anonymous access (basic rate limits)
2025-10-03 09:30:04,420 - INFO - Planetary Computer authentication status: Anonymous


In [ ]:
def search_sentinel2_scenes(
    bbox: List[float],
    date_range: str,
    cloud_cover_max: float = 20,
    limit: int = 10
) -> List:
    """
    STANDARD NUMPY WAY OF DEFINING PARAMETERS
    Search Sentinel-2 Level 2A scenes using STAC API.

    Parameters
    ----------
    bbox : List[float]
        Bounding box as [west, south, east, north] in WGS84
    date_range : str
        ISO date range: "YYYY-MM-DD/YYYY-MM-DD"
    cloud_cover_max : float
        Maximum cloud cover percentage
    limit : int
        Maximum scenes to return

    Returns
    -------
    List[pystac.Item]
        List of STAC items sorted by cloud cover (ascending)
    """
    catalog = Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace
    )

    search_params = {
        "collections": ["sentinel-2-l2a"],
        "bbox": bbox,
        "datetime": date_range,
        "query": {"eo:cloud_cover": {"lt": cloud_cover_max}},
        "limit": limit
    }

    search_results = catalog.search(**search_params)
    items = list(search_results.items())

    # Sort by cloud cover (best quality first)
    items.sort(key=lambda x: x.properties.get('eo:cloud_cover', 100))

    logger.info(f"Found {len(items)} Sentinel-2 scenes (cloud cover < {cloud_cover_max}%)")
    return items

In [25]:
def load_sentinel2_bands(
    item,
    bands: List[str] = ['B04', 'B03', 'B02', 'B08'],
    subset_bbox: Optional[List[float]] = None,
    max_retries: int = 3
) -> Dict[str, Union[np.ndarray, str]]:
    """
    Load Sentinel-2 bands with optional spatial subsetting.

    Parameters
    ----------
    item : pystac.Item
        STAC item representing the satellite scene
    bands : List[str]
        Spectral bands to load
    subset_bbox : Optional[List[float]]
        Spatial subset as [west, south, east, north] in WGS84
    max_retries : int
        Number of retry attempts per band

    Returns
    -------
    Dict[str, Union[np.ndarray, str]]
        Band arrays plus georeferencing metadata
    """
    from rasterio.windows import from_bounds
    from rasterio.warp import transform_bounds

    band_data = {}
    successful_bands = []
    failed_bands = []

    for band_name in bands:
        if band_name not in item.assets:
            failed_bands.append(band_name)
            continue

        asset_url = item.assets[band_name].href

        # Retry logic with exponential backoff
        for attempt in range(max_retries):
            try:
                # URL signing for authenticated access
                signed_url = pc.sign(asset_url)

                # Memory-efficient loading with rasterio
                with rasterio.open(signed_url) as src:
                    # Validate data source
                    if src.width == 0 or src.height == 0:
                        raise ValueError(f"Invalid raster dimensions: {src.width}x{src.height}")

                    if subset_bbox:
                        # Intelligent subsetting with CRS transformation
                        try:
                            # Transform bbox to source CRS if needed
                            if src.crs != rasterio.crs.CRS.from_epsg(4326):
                                subset_bbox_src_crs = transform_bounds(
                                    rasterio.crs.CRS.from_epsg(4326), src.crs, *subset_bbox
                                )
                            else:
                                subset_bbox_src_crs = subset_bbox

                            # Calculate reading window
                            window = from_bounds(*subset_bbox_src_crs, src.transform)

                            # Ensure window is within raster bounds
                            window = window.intersection(
                                rasterio.windows.Window(0, 0, src.width, src.height)
                            )

                            if window.width > 0 and window.height > 0:
                                data = src.read(1, window=window)
                                transform = src.window_transform(window)
                                bounds = rasterio.windows.bounds(window, src.transform)
                                if src.crs != rasterio.crs.CRS.from_epsg(4326):
                                    bounds = transform_bounds(src.crs, rasterio.crs.CRS.from_epsg(4326), *bounds)
                            else:
                                # Fall back to full scene
                                data = src.read(1)
                                transform = src.transform
                                bounds = src.bounds
                        except Exception:
                            # Fall back to full scene on subset error
                            data = src.read(1)
                            transform = src.transform
                            bounds = src.bounds
                    else:
                        # Load full scene
                        data = src.read(1)
                        transform = src.transform
                        bounds = src.bounds

                    if data.size == 0:
                        raise ValueError("Loaded data has zero size")

                    # Store band data and metadata
                    band_data[band_name] = data
                    if 'transform' not in band_data:
                        band_data.update({
                            'transform': transform,
                            'crs': src.crs,
                            'bounds': bounds,
                            'scene_id': item.id,
                            'date': item.properties['datetime'].split('T')[0]
                        })

                    successful_bands.append(band_name)
                    break

            except Exception as e:
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)  # Exponential backoff
                    continue
                else:
                    failed_bands.append(band_name)
                    logger.warning(f"Failed to load band {band_name}: {str(e)[:50]}")
                    break

    # Validate results
    if len(successful_bands) == 0:
        raise Exception(f"Failed to load any bands from scene {item.id}")

    if failed_bands:
        logger.warning(f"Failed to load {len(failed_bands)} bands: {failed_bands}")

    logger.info(f"Successfully loaded {len(successful_bands)} bands: {successful_bands}")
    return band_data

In [10]:
def search_STAC_scenes(
    bbox: list,
    date_range: str,
    cloud_cover_max: float = 100.0,
    limit: int = 10,
    collection: str = "sentinel-2-l2a",
    stac_url: str = "https://planetarycomputer.microsoft.com/api/stac/v1",
    client_modifier=None,
    extra_query: dict = None
) -> list:
    """
    General-purpose function to search STAC scenes using a STAC API.

    Parameters
    ----------
    bbox : List[float]
        Bounding box as [west, south, east, north] in WGS84
    date_range : str
        ISO date range: "YYYY-MM-DD/YYYY-MM-DD"
    cloud_cover_max : float, optional
        Maximum cloud cover percentage (default: 100.0)
    limit : int, optional
        Maximum scenes to return (default: 10)
    collection : str, optional
        STAC collection name (default: "sentinel-2-l2a")
    stac_url : str, optional
        STAC API endpoint URL (default: MPC STAC)
    client_modifier : callable, optional
        Optional function to modify the STAC client (e.g., for auth)
    extra_query : dict, optional
        Additional query parameters for the search

    Returns
    -------
    List[pystac.Item]
        List of STAC items sorted by cloud cover (ascending, if available).

    Examples
    --------
    >>> # Search for Sentinel-2 scenes (default) on the Microsoft Planetary Computer (default) 
    >>> # over a bounding box in Oregon in January 2022
    >>> bbox = [-123.5, 45.0, -122.5, 46.0]
    >>> date_range = "2022-01-01/2022-01-31"
    >>> items = search_STAC_scenes(bbox, date_range, cloud_cover_max=10, limit=5)

    >>> # Search for Landsat 8 scenes from a different STAC endpoint
    >>> landsat_url = "https://earth-search.aws.element84.com/v1"
    >>> items = search_STAC_scenes(
    ...     bbox,
    ...     "2021-06-01/2021-06-30",
    ...     collection="landsat-8-c2-l2",
    ...     stac_url=landsat_url,
    ...     cloud_cover_max=20,
    ...     limit=3
    ... )

    >>> # Add an extra query to filter by platform
    >>> items = search_STAC_scenes(
    ...     bbox,
    ...     date_range,
    ...     extra_query={"platform": {"eq": "sentinel-2b"}}
    ... )
    """
    # Open the STAC client, with optional modifier (e.g., for MPC auth)
    if client_modifier is not None:
        catalog = Client.open(stac_url, modifier=client_modifier)
    else:
        catalog = Client.open(stac_url)

    # Build query parameters
    search_params = {
        "collections": [collection],
        "bbox": bbox,
        "datetime": date_range,
        "limit": limit
    }

    # Add cloud cover filter if present
    if cloud_cover_max < 100.0:
        search_params["query"] = {"eo:cloud_cover": {"lt": cloud_cover_max}}
    if extra_query:
        # Merge extra_query into search_params['query']
        if "query" not in search_params:
            search_params["query"] = {}
        search_params["query"].update(extra_query)

    search_results = catalog.search(**search_params)
    items = list(search_results.items())

    # Sort by cloud cover if available
    items.sort(key=lambda x: x.properties.get('eo:cloud_cover', 100))

    logger.info(
        f"Found {len(items)} scenes in collection '{collection}' (cloud cover < {cloud_cover_max}%)"
    )
    return items

catelogs give you url, need conservative loading strategies to get data onto the machine 

get list of bands, pull subset of it 

list pystack.item -- but that is a type that no type engine knows what it is, type checkers wouldnt understand what it is 


for quality assurance pull down to local machine, make sure pixels align 

more efficent way load these functions into the .py file 

In [12]:
# just wanted middle 50% of the image so hacky way to do that

def get_subset_from_scene(
    item,
    x_range: Tuple[float, float] = (25, 75),
    y_range: Tuple[float, float] = (25, 75),
) -> List[float]:
    """
    Intelligent spatial subsetting using percentage-based coordinates.

    This approach provides several advantages:
    1. Resolution Independence: Works regardless of scene size or pixel resolution
    2. Reproducibility: Same percentage always gives same relative location
    3. Scalability: Easy to create systematic grids for batch processing
    4. Adaptability: Can adjust subset size based on scene characteristics

    Parameters
    ----------
    item : pystac.Item
        STAC item containing scene geometry
    x_range : Tuple[float, float]
        Longitude percentage range (0-100)
    y_range : Tuple[float, float]
        Latitude percentage range (0-100)

    Returns
    -------
    List[float]
        Subset bounding box [west, south, east, north] in WGS84

    Design Pattern: Template Method with Spatial Reasoning
    - Provides consistent interface for varied spatial operations
    - Encapsulates coordinate system complexity
    - Enables systematic spatial sampling strategies
    """
    # Extract scene geometry from STAC metadata
    scene_bbox = item.bbox  # [west, south, east, north]

    # Input validation for percentage ranges
    if not (0 <= x_range[0] < x_range[1] <= 100):
        raise ValueError(
            f"Invalid x_range: {x_range}. Must be (min, max) with 0 <= min < max <= 100"
        )
    if not (0 <= y_range[0] < y_range[1] <= 100):
        raise ValueError(
            f"Invalid y_range: {y_range}. Must be (min, max) with 0 <= min < max <= 100"
        )

    # Calculate scene dimensions in geographic coordinates
    scene_width = scene_bbox[2] - scene_bbox[0]  # east - west
    scene_height = scene_bbox[3] - scene_bbox[1]  # north - south

    # Convert percentages to geographic coordinates
    west = scene_bbox[0] + (x_range[0] / 100.0) * scene_width
    east = scene_bbox[0] + (x_range[1] / 100.0) * scene_width
    south = scene_bbox[1] + (y_range[0] / 100.0) * scene_height
    north = scene_bbox[1] + (y_range[1] / 100.0) * scene_height

    subset_bbox = [west, south, east, north]

    # Calculate subset metrics for reporting
    subset_area_percent = (
        (x_range[1] - x_range[0]) * (y_range[1] - y_range[0])
    ) / 100.0

    logger.info("📐 Calculated subset from scene bounds:")
    logger.info(
        "   Scene bbox: [%.4f, %.4f, %.4f, %.4f]",
        scene_bbox[0], scene_bbox[1], scene_bbox[2], scene_bbox[3]
    )
    logger.info(
        "   Subset bbox: [%.4f, %.4f, %.4f, %.4f]",
        west, south, east, north
    )
    logger.info(
        "   X range: %s%%-%s%%, Y range: %s%%-%s%%",
        x_range[0], x_range[1], y_range[0], y_range[1]
    )
    logger.info(
        "   Subset area: %.1f%% of original scene",
        subset_area_percent
    )

    return subset_bbox

In [13]:
def get_scene_info(item):
    """
    Extract comprehensive scene characteristics for adaptive processing.

    Parameters
    ----------
    item : pystac.Item
        STAC item to analyze

    Returns
    -------
    Dict
        Scene characteristics including dimensions and geographic metrics

    Design Pattern: Information Expert
    - Centralizes scene analysis logic
    - Provides basis for adaptive processing decisions
    - Enables consistent scene characterization across workflows
    """
    bbox = item.bbox
    width_deg = bbox[2] - bbox[0]
    height_deg = bbox[3] - bbox[1]

    # Approximate conversion to kilometers (suitable for most latitudes)
    center_lat = (bbox[1] + bbox[3]) / 2
    width_km = width_deg * 111 * np.cos(np.radians(center_lat))
    height_km = height_deg * 111

    info = {
        "scene_id": item.id,
        "date": item.properties["datetime"].split("T")[0],
        "bbox": bbox,
        "width_deg": width_deg,
        "height_deg": height_deg,
        "width_km": width_km,
        "height_km": height_km,
        "area_km2": width_km * height_km,
        "center_lat": center_lat,
        "center_lon": (bbox[0] + bbox[2]) / 2,
    }

    return info

In [14]:
# Connect to STAC catalog
try:
    catalog = Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace
    )

    logger.info("Connected to Planetary Computer STAC API")

    # Get catalog information
    try:
        catalog_info = catalog.get_self()
        logger.info(f"Catalog: {catalog_info.title}")
    except Exception:
        logger.info("Basic connection successful")

    # Explore key satellite datasets
    satellite_collections = {
        'sentinel-2-l2a': 'Sentinel-2 Level 2A (10m optical)',
        'landsat-c2-l2': 'Landsat Collection 2 Level 2 (30m optical)',
        'sentinel-1-grd': 'Sentinel-1 SAR (radar)',
        'naip': 'NAIP (1m aerial imagery)'
    }

    available_collections = []
    for collection_id, description in satellite_collections.items():
        try:
            collection = catalog.get_collection(collection_id)
            available_collections.append(collection_id)
            logger.info(f"Available: {description}")
        except Exception:
            logger.warning(f"Not accessible: {description}")

    logger.info(f"Accessible collections: {len(available_collections)}/{len(satellite_collections)}")

except Exception as e:
    logger.error(f"\n❌ STAC connection failed: {str(e)}")
    logger.info(f"\n🔧 Troubleshooting steps:")
    logger.info(f"   1. Verify internet connectivity")
    logger.info(f"   2. Check Planetary Computer API status: https://planetarycomputer.microsoft.com/")
    logger.info(f"   3. Ensure pystac-client is installed: pip install pystac-client")
    logger.info(f"   4. Verify planetary-computer package: pip install planetary-computer")
    logger.info(f"   5. Try again in a few minutes (temporary API issues)")

2025-10-03 09:54:39,129 - INFO - Connected to Planetary Computer STAC API
2025-10-03 09:54:39,131 - INFO - Basic connection successful
2025-10-03 09:54:40,318 - INFO - Available: Sentinel-2 Level 2A (10m optical)
2025-10-03 09:54:40,530 - INFO - Available: Landsat Collection 2 Level 2 (30m optical)
2025-10-03 09:54:40,725 - INFO - Available: Sentinel-1 SAR (radar)
2025-10-03 09:54:40,918 - INFO - Available: NAIP (1m aerial imagery)
2025-10-03 09:54:40,920 - INFO - Accessible collections: 4/4


In [15]:
# Step 3A: Define Area of Interest with Geographic Reasoning
# Primary study area: Santa Barbara Region
# Coordinates chosen to encompass the greater Santa Barbara County coastal region
santa_barbara_bbox = [-120.5, 34.3, -119.5, 34.7]  # [west, south, east, north]

# Import required libraries for spatial calculations
from shapely.geometry import box
import os

# Create geometry object for area calculations
aoi_geom = box(*santa_barbara_bbox)

# Calculate basic spatial metrics
area_degrees = aoi_geom.area
# Approximate conversion to kilometers (valid for mid-latitudes)
center_lat = (santa_barbara_bbox[1] + santa_barbara_bbox[3]) / 2
lat_correction = np.cos(np.radians(center_lat))
area_km2 = area_degrees * (111.32 ** 2) * lat_correction  # 1 degree ≈ 111.32 km

logger.info(f"\n📊 AOI Spatial Characteristics:")
logger.info(f"   📍 Region: Santa Barbara County Coastal Region")
logger.info(f"   🗺️ Bounding box: {santa_barbara_bbox}")
logger.info(f"   📐 Dimensions: {(santa_barbara_bbox[2] - santa_barbara_bbox[0]):.2f}° × {(santa_barbara_bbox[3] - santa_barbara_bbox[1]):.2f}°")
logger.info(f"   📏 Approximate area: {area_km2:.0f} km²")
logger.info(f"   🏙️ Population: ~450,000 (Santa Barbara County)")

# Provide alternative study areas for different research interests
logger.info(f"\n🌐 Alternative AOI Options for Different Study Objectives:")
alternative_aois = {
    "San Francisco Bay Area": {
        "bbox": [-122.5, 37.3, -121.8, 38.0],
        "focus": "Urban growth, water dynamics, mixed land use",
        "challenges": "Fog and cloud cover in summer"
    },
    "Los Angeles Basin": {
        "bbox": [-118.7, 33.7, -118.1, 34.3],
        "focus": "Urban heat islands, air quality, sprawl patterns",
        "challenges": "Frequent clouds, complex topography"
    },
    "Central Valley Agriculture": {
        "bbox": [-121.5, 36.0, -120.0, 37.5],
        "focus": "Crop monitoring, irrigation patterns, drought",
        "challenges": "Seasonal variations, haze"
    },
    "Channel Islands": {
        "bbox": [-120.5, 33.9, -119.0, 34.1],
        "focus": "Island ecology, marine-terrestrial interface, conservation",
        "challenges": "Marine layer, limited ground truth"
    }
}

for region, info in alternative_aois.items():
    bbox = info["bbox"]
    area_alt = ((bbox[2] - bbox[0]) * (bbox[3] - bbox[1]) *
                (111.32 ** 2) * np.cos(np.radians((bbox[1] + bbox[3]) / 2)))
    logger.info(f"   🗺️ {region}: {info['bbox']} ({area_alt:.0f} km²)")
    logger.info(f"      🎯 Research focus: {info['focus']}")
    logger.info(f"      ⚠️ Considerations: {info['challenges']}")

logger.info(f"\n💡 Pro Tip: Choose AOI based on:")
logger.info(f"   1. Research objectives and required spatial resolution")
logger.info(f"   2. Data availability and typical cloud cover patterns")
logger.info(f"   3. Computational resources and processing time constraints")
logger.info(f"   4. Ecological or administrative boundary alignment")

2025-10-03 09:55:23,170 - INFO - 
📊 AOI Spatial Characteristics:
2025-10-03 09:55:23,172 - INFO -    📍 Region: Santa Barbara County Coastal Region
2025-10-03 09:55:23,173 - INFO -    🗺️ Bounding box: [-120.5, 34.3, -119.5, 34.7]
2025-10-03 09:55:23,173 - INFO -    📐 Dimensions: 1.00° × 0.40°
2025-10-03 09:55:23,175 - INFO -    📏 Approximate area: 4085 km²
2025-10-03 09:55:23,177 - INFO -    🏙️ Population: ~450,000 (Santa Barbara County)
2025-10-03 09:55:23,177 - INFO - 
🌐 Alternative AOI Options for Different Study Objectives:
2025-10-03 09:55:23,179 - INFO -    🗺️ San Francisco Bay Area: [-122.5, 37.3, -121.8, 38.0] (4808 km²)
2025-10-03 09:55:23,180 - INFO -       🎯 Research focus: Urban growth, water dynamics, mixed land use
2025-10-03 09:55:23,181 - INFO -       ⚠️ Considerations: Fog and cloud cover in summer
2025-10-03 09:55:23,182 - INFO -    🗺️ Los Angeles Basin: [-118.7, 33.7, -118.1, 34.3] (3698 km²)
2025-10-03 09:55:23,182 - INFO -       🎯 Research focus: Urban heat islands,

In [16]:
# Step 3B: Create Interactive Map with Multiple Basemap Options
# Calculate map center for optimal display
center_lat = (santa_barbara_bbox[1] + santa_barbara_bbox[3]) / 2
center_lon = (santa_barbara_bbox[0] + santa_barbara_bbox[2]) / 2

# Initialize folium map with appropriate zoom level
# Zoom level chosen to show entire AOI while maintaining detail
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=9,  # Optimal for metropolitan area viewing
    tiles='OpenStreetMap'
)

# Add diverse basemap options for different analysis contexts
basemap_options = {
    'CartoDB positron': {
        'layer': folium.TileLayer('CartoDB positron', name='Clean Basemap'),
        'use_case': 'Data overlay visualization, presentations'
    },
    'CartoDB dark_matter': {
        'layer': folium.TileLayer('CartoDB dark_matter', name='Dark Theme'),
        'use_case': 'Night mode, reducing eye strain'
    },
    'Esri World Imagery': {
        'layer': folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='Esri', name='Satellite Imagery', overlay=False, control=True
        ),
        'use_case': 'Ground truth validation, visual interpretation'
    },
    'OpenTopoMap': {
        'layer': folium.TileLayer(
            tiles='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
            name='Topographic (OpenTopoMap)',
            attr='Map data: © OpenStreetMap contributors, SRTM | Map style: © OpenTopoMap (CC-BY-SA)',
            overlay=False,
            control=True
        ),
        'use_case': 'Elevation context, watershed analysis'
    }
}

logger.info(f"   📚 Adding {len(basemap_options)} basemap options:")
for name, info in basemap_options.items():
    info['layer'].add_to(m)
    logger.info(f"     • {name}: {info['use_case']}")

# Add AOI boundary with informative styling
aoi_bounds = [[santa_barbara_bbox[1], santa_barbara_bbox[0]],  # southwest corner
              [santa_barbara_bbox[3], santa_barbara_bbox[2]]]  # northeast corner

folium.Rectangle(
    bounds=aoi_bounds,
    color='red',
    weight=3,
    fill=True,
    fillOpacity=0.1,
    popup=folium.Popup(
        f"""
        <div style="font-family: Arial; width: 300px;">
        <h4>📊 Study Area Details</h4>
        <b>Region:</b> Santa Barbara County Coastal Region<br>
        <b>Coordinates:</b> {santa_barbara_bbox}<br>
        <b>Area:</b> {area_km2:.0f} km²<br>
        <b>Purpose:</b> Geospatial AI Training<br>
        <b>Data Type:</b> Sentinel-2 Optical<br>
        </div>
        """,
        max_width=350
    ),
    tooltip="Study Area Boundary - Click for details"
).add_to(m)

# Add geographic reference points with contextual information
reference_locations = [
    {
        "name": "Santa Barbara",
        "coords": [34.4208, -119.6982],
        "description": "Coastal city, urban-wildland interface",
        "icon": "building",
        "color": "blue"
    },
    {
        "name": "UCSB",
        "coords": [34.4140, -119.8489],
        "description": "University campus, research facilities",
        "icon": "graduation-cap",
        "color": "green"
    },
    {
        "name": "Goleta",
        "coords": [34.4358, -119.8276],
        "description": "Tech corridor, agricultural transition zone",
        "icon": "microchip",
        "color": "purple"
    },
    {
        "name": "Montecito",
        "coords": [34.4358, -119.6376],
        "description": "Wildfire-prone, high-value urban area",
        "icon": "fire",
        "color": "red"
    }
]

logger.info(f"Adding {len(reference_locations)} geographic reference points")
for location in reference_locations:
    logger.debug(f"{location['name']}: {location['description']}")

    folium.Marker(
        location=location["coords"],
        popup=folium.Popup(
            f"""
            <div style="font-family: Arial;">
            <h4>{location['name']}</h4>
            <b>Coordinates:</b> {location['coords'][0]:.4f}, {location['coords'][1]:.4f}<br>
            <b>Context:</b> {location['description']}<br>
            <b>Role in Analysis:</b> Geographic reference point
            </div>
            """,
            max_width=250
        ),
        tooltip=f"{location['name']} - {location['description']}",
        icon=folium.Icon(
            color=location['color'],
            icon=location['icon'],
            prefix='fa'
        )
    ).add_to(m)

# Add measurement and interaction tools for analysis
logger.info("Adding interactive analysis tools")

# Measurement tool for distance/area calculations
from folium.plugins import MeasureControl
measure_control = MeasureControl(
    primary_length_unit='kilometers',
    primary_area_unit='sqkilometers',
    secondary_length_unit='miles',
    secondary_area_unit='sqmiles'
)
m.add_child(measure_control)
logger.debug("Added measurement tool for distance/area calculations")

# Fullscreen capability for detailed examination
from folium.plugins import Fullscreen
Fullscreen(
    position='topright',
    title='Full Screen Mode',
    title_cancel='Exit Full Screen',
    force_separate_button=True
).add_to(m)
logger.debug("Added fullscreen mode capability")

# Layer control for basemap switching
layer_control = folium.LayerControl(
    position='topright',
    collapsed=False
)
layer_control.add_to(m)
logger.debug("Added layer control for basemap switching")

logger.info("Interactive map created with comprehensive spatial context")

# Display the map
m

2025-10-03 09:56:31,705 - INFO -    📚 Adding 4 basemap options:
2025-10-03 09:56:31,707 - INFO -      • CartoDB positron: Data overlay visualization, presentations
2025-10-03 09:56:31,708 - INFO -      • CartoDB dark_matter: Night mode, reducing eye strain
2025-10-03 09:56:31,708 - INFO -      • Esri World Imagery: Ground truth validation, visual interpretation
2025-10-03 09:56:31,708 - INFO -      • OpenTopoMap: Elevation context, watershed analysis
2025-10-03 09:56:31,710 - INFO - Adding 4 geographic reference points
2025-10-03 09:56:31,711 - INFO - Adding interactive analysis tools
2025-10-03 09:56:31,712 - INFO - Interactive map created with comprehensive spatial context


In [17]:
# Step 4A: Implement Robust Multi-Strategy Scene Discovery
from datetime import datetime, timedelta

# Strategy 1: Dynamic temporal window based on current date
current_date = datetime.now()
logger.info(f"Calculating optimal temporal search windows (current date: {current_date.strftime('%Y-%m-%d')})")

# Define multiple search strategies with different trade-offs
# Each strategy balances data quality against data availability
search_strategies = [
    {
        "name": "Optimal Quality",
        "date_range": "2024-06-01/2024-09-30",
        "cloud_max": 20,
        "description": "Recent summer data with excellent atmospheric conditions",
        "priority": "Best for analysis quality",
        "trade_offs": "May have limited availability"
    },
    {
        "name": "Good Quality",
        "date_range": "2024-03-01/2024-08-31",
        "cloud_max": 35,
        "description": "Extended seasonal window with good conditions",
        "priority": "Balance of quality and availability",
        "trade_offs": "Some atmospheric interference"
    },
    {
        "name": "Acceptable Quality",
        "date_range": "2023-09-01/2024-11-30",
        "cloud_max": 50,
        "description": "Broader temporal and quality window",
        "priority": "Reliable data availability",
        "trade_offs": "May require additional preprocessing"
    },
    {
        "name": "Fallback Option",
        "date_range": "2023-01-01/2024-12-31",
        "cloud_max": 75,
        "description": "Maximum temporal window, relaxed quality constraints",
        "priority": "Guaranteed data access",
        "trade_offs": "Significant cloud contamination possible"
    }
]

logger.info(f"Defined {len(search_strategies)} search strategies")
for i, strategy in enumerate(search_strategies, 1):
    logger.debug(f"Strategy {i}: {strategy['name']} - {strategy['description']}")

# Execute search strategies in order of preference
scenes = []
successful_strategy = None

for i, strategy in enumerate(search_strategies, 1):
    logger.info(f"Executing Strategy {i}: {strategy['name']} (dates: {strategy['date_range']}, cloud < {strategy['cloud_max']}%)")

    try:
        # Use our optimized search function with current strategy parameters
        temp_scenes = search_sentinel2_scenes(
            bbox=santa_barbara_bbox,
            date_range=strategy["date_range"],
            cloud_cover_max=strategy["cloud_max"],
            limit=100  # Generous limit for selection flexibility
        )

        if temp_scenes:
            scenes = temp_scenes
            successful_strategy = strategy
            logger.info(f"SUCCESS! Found {len(scenes)} qualifying scenes with {strategy['name']} strategy")
            break
        else:
            logger.warning(f"No scenes found with {strategy['name']} strategy, proceeding to next")

    except Exception as e:
        logger.warning(f"Search execution failed for {strategy['name']}: {str(e)[:80]}")
        continue

# Validate search results and provide detailed feedback
if not scenes:
    logger.error(f"Scene discovery failed after trying all {len(search_strategies)} strategies")
    logger.info("Diagnostic steps: 1) Check network connectivity, 2) Verify API status, 3) Confirm AOI coverage, 4) Try broader date ranges, 5) Check authentication")
    raise Exception("Critical failure in scene discovery. Review diagnostic steps and retry.")

logger.info(f"Scene discovery completed: {successful_strategy['name']} strategy found {len(scenes)} scenes (attempt {search_strategies.index(successful_strategy) + 1}/{len(search_strategies)})")

2025-10-03 09:58:10,385 - INFO - Calculating optimal temporal search windows (current date: 2025-10-03)
2025-10-03 09:58:10,388 - INFO - Defined 4 search strategies
2025-10-03 09:58:10,389 - INFO - Executing Strategy 1: Optimal Quality (dates: 2024-06-01/2024-09-30, cloud < 20%)
2025-10-03 09:58:14,476 - INFO - Found 40 Sentinel-2 scenes (cloud cover < 20%)
2025-10-03 09:58:14,477 - INFO - SUCCESS! Found 40 qualifying scenes with Optimal Quality strategy
2025-10-03 09:58:14,478 - INFO - Scene discovery completed: Optimal Quality strategy found 40 scenes (attempt 1/4)


In [23]:
def test_subset_functionality(item):
    """
    Automated quality assurance for data loading pipelines.

    This testing approach demonstrates:
    1. Smoke Testing: Verify basic functionality before full processing
    2. Representative Sampling: Test with manageable data subset
    3. Error Detection: Identify issues early in processing pipeline
    4. Performance Validation: Ensure acceptable loading performance

    Parameters
    ----------
    item : pystac.Item
        STAC item to test

    Returns
    -------
    bool
        True if subset functionality is working correctly

    Design Pattern: Chain of Responsibility for Quality Assurance
    - Implements systematic testing hierarchy
    - Provides early failure detection
    - Validates core functionality before expensive operations
    """
    logger.info(f"🧪 Testing subset functionality...")

    try:
        # Test with small central area (minimal data transfer)
        test_bbox = get_subset_from_scene(item, x_range=(40, 60), y_range=(40, 60))

        # Load minimal data for testing
        test_data = load_sentinel2_bands(
            item,
            bands=["B04"],  # Single band reduces test time
            subset_bbox=test_bbox,
            max_retries=2,
        )

        if "B04" in test_data:
            shape = test_data["B04"].shape
            has_data = test_data["B04"].size > 0
            logger.info(
                f"   ✅ Subset test successful: {shape} pixels, {test_data['B04'].size} total"
            )
            return True
        else:
            logger.error(f"   ❌ Subset test failed: no data returned")
            return False

    except Exception as e:
        logger.error(f"   ❌ Subset test failed: {str(e)[:50]}...")
        return False

In [26]:
# Step 4B: Intelligent Scene Selection Based on Multiple Quality Criteria
# Sort scenes by multiple quality criteria
# Primary: cloud cover (lower is better)
# Secondary: date (more recent is better)
scenes_with_scores = []

logger.info(f"Evaluating {len(scenes)} candidate scenes for quality assessment")
for scene in scenes:
    props = scene.properties

    # Extract key quality metrics
    cloud_cover = props.get('eo:cloud_cover', 100)
    date_str = props.get('datetime', '').split('T')[0]
    scene_date = datetime.strptime(date_str, '%Y-%m-%d')
    days_old = (current_date - scene_date).days

    # Calculate composite quality score (lower is better)
    # Weight factors: cloud cover (70%), recency (30%)
    cloud_score = cloud_cover  # 0-100 scale
    recency_score = min(days_old / 30, 100)  # Normalize to 0-100, cap at 100
    quality_score = (0.7 * cloud_score) + (0.3 * recency_score)

    scene_info = {
        'scene': scene,
        'date': date_str,
        'cloud_cover': cloud_cover,
        'days_old': days_old,
        'quality_score': quality_score,
        'tile_id': props.get('sentinel:grid_square', 'Unknown'),
        'platform': props.get('platform', 'Sentinel-2')
    }

    scenes_with_scores.append(scene_info)

# Sort by quality score (best first)
scenes_with_scores.sort(key=lambda x: x['quality_score'])

# Display top candidates for educational purposes
logger.info("Top 5 Scene Candidates (ranked by quality score):")
for i, scene_info in enumerate(scenes_with_scores[:5], 1):
    logger.debug(f"{i}. {scene_info['date']} - Cloud: {scene_info['cloud_cover']:.1f}%, Age: {scene_info['days_old']} days, Score: {scene_info['quality_score']:.1f}")
    if i == 1:
        logger.info(f"Selected optimal scene: {scene_info['date']}")

# Select the best scene
best_scene_info = scenes_with_scores[0]
best_scene = best_scene_info['scene']

logger.info(f"Optimal scene selected: {best_scene_info['date']} ({best_scene_info['cloud_cover']:.1f}% cloud cover, {best_scene_info['platform']}, Tile: {best_scene_info['tile_id']})")

# Validate scene completeness for required analysis
logger.info("Validating scene data completeness")
required_bands = ['B02', 'B03', 'B04', 'B08']  # Blue, Green, Red, NIR
available_bands = list(best_scene.assets.keys())
spectral_bands = [b for b in available_bands if b.startswith('B') and len(b) <= 3]

logger.debug(f"Available spectral bands: {len(spectral_bands)}, Required: {required_bands}")

missing_bands = [b for b in required_bands if b not in available_bands]
if missing_bands:
    logger.warning(f"Missing critical bands: {missing_bands} - this may limit analysis capabilities")

    # Check for alternative bands
    alternative_mapping = {'B02': 'blue', 'B03': 'green', 'B04': 'red', 'B08': 'nir'}
    alternatives_found = []
    for missing in missing_bands:
        alt_name = alternative_mapping.get(missing, missing.lower())
        if alt_name in available_bands:
            alternatives_found.append((missing, alt_name))

    if alternatives_found:
        logger.info(f"Found alternative band names: {alternatives_found}")
else:
    logger.info("All required bands available")

# Additional quality checks
extra_bands = [b for b in spectral_bands if b not in required_bands]
if extra_bands:
    logger.debug(f"Bonus bands available: {extra_bands[:5]}{'...' if len(extra_bands) > 5 else ''} (enable advanced spectral analysis)")

logger.info("Scene validation complete - ready for data loading")

# Quick connectivity test using our helper function
logger.info("Performing pre-flight connectivity test")
connectivity_test = test_subset_functionality(best_scene)

if connectivity_test:
    logger.info("Data access confirmed - all systems ready")
else:
    logger.warning("Connectivity issues detected - will attempt full download with fallback mechanisms")

2025-10-03 10:12:26,887 - INFO - Evaluating 40 candidate scenes for quality assessment
2025-10-03 10:12:26,890 - INFO - Top 5 Scene Candidates (ranked by quality score):
2025-10-03 10:12:26,891 - INFO - Selected optimal scene: 2024-08-13
2025-10-03 10:12:26,892 - INFO - Optimal scene selected: 2024-08-13 (0.0% cloud cover, Sentinel-2B, Tile: Unknown)
2025-10-03 10:12:26,894 - INFO - Validating scene data completeness
2025-10-03 10:12:26,895 - INFO - All required bands available
2025-10-03 10:12:26,897 - INFO - Scene validation complete - ready for data loading
2025-10-03 10:12:26,898 - INFO - Performing pre-flight connectivity test
2025-10-03 10:12:26,899 - INFO - 🧪 Testing subset functionality...
2025-10-03 10:12:26,899 - INFO - 📐 Calculated subset from scene bounds:
2025-10-03 10:12:26,900 - INFO -    Scene bbox: [-120.2952, 34.2097, -119.0654, 35.2250]
2025-10-03 10:12:26,901 - INFO -    Subset bbox: [-119.8033, 34.6158, -119.5573, 34.8189]
2025-10-03 10:12:26,901 - INFO -    X rang